# Algoritmo Apriori

## Importar las bibliotecas necesarias

Esta parte del código importa las bibliotecas necesarias para implementar el algoritmo Apriori y analizar reglas de asociación utilizando Python. 

- `mlxtend.preprocessing.TransactionEncoder`: Convierte los datos en una matriz binaria, donde cada fila representa una transacción y cada columna representa un ítem, y marca con 1 si el ítem está presente en la transacción y 0 si no lo está.

- `mlxtend.frequent_patterns.apriori`: Esta función implementa el algoritmo Apriori para encontrar conjuntos de ítems frecuentes en los datos de transacción.

- `mlxtend.frequent_patterns.association_rules`: Esta función genera reglas de asociación a partir de los conjuntos de ítems frecuentes encontrados por el algoritmo Apriori.

- `pandas`: Esta biblioteca se utiliza para manipular y analizar datos en formato tabular, como un DataFrame, lo que facilita el análisis y la visualización de los resultados de las reglas de asociación.

In [1]:
# Importar las bibliotecas necesarias
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import pandas as pd

## Definir las transacciones

En esta parte del código se definen las transacciones de un supermercado. Cada tupla dentro de la lista `transacciones` representa una compra realizada por un cliente en el supermercado. Cada elemento dentro de la tupla corresponde a un artículo comprado por el cliente en esa transacción específica. 

In [2]:
# Definir las transacciones
# Cada tupla representa una transacción individual en un supermercado
transacciones = [
    ("LECHE", "PAN", "HUEVO"),
    ("CEREAL", "LECHE", "CREMA", "HUEVO"),
    ("LECHE", "CREMA", "PAN", "HUEVO")
]

## Crear un codificador de transacciones

En esta parte del código, se está creando un objeto de la clase `TransactionEncoder`. Esta clase es parte de la biblioteca `mlxtend.preprocessing` y se utiliza para convertir las transacciones, representadas como listas de elementos, en una matriz binaria. Esta matriz binaria es una representación necesaria para el algoritmo Apriori, ya que se usara proximamente.

In [3]:
# Crear un codificador de transacciones
# Este objeto se utiliza para transformar las transacciones en una matriz de booleanos
te = TransactionEncoder()

## Ajustar y transformar las transacciones en una matriz de booleanos
Se utiliza el objeto creado para transformar las transacciones representadas como listas de elementos en una matriz binaria. 

- Primero, se utiliza el método `fit(transacciones)` para ajustar el codificador a las transacciones dadas, lo que significa que el codificador aprende los diferentes elementos únicos presentes en las transacciones.
  
- Luego, se utiliza el método `transform(transacciones)` para transformar las transacciones en una matriz binaria, donde cada fila representa una transacción y cada columna representa un elemento. Si un elemento está presente en una transacción, se marca con 1; de lo contrario, se marca con 0.

La variable `te_ary` contendrá esta matriz binaria, que se utilizará como entrada para el algoritmo Apriori para encontrar conjuntos de ítems frecuentes en los datos transaccionales.

In [4]:
# Ajustar y transformar las transacciones en una matriz de booleanos
# Cada fila de la matriz representa una transacción y cada columna representa un producto
# Un valor True en una celda indica que el producto correspondiente fue comprado en la transacción correspondiente
te_ary = te.fit(transacciones).transform(transacciones)
print(f"Cereal ,Crema ,Huevo ,Leche ,Pan \n")
print(te_ary)

Cereal ,Crema ,Huevo ,Leche ,Pan 

[[False False  True  True  True]
 [ True  True  True  True False]
 [False  True  True  True  True]]


## Crear un DataFrame de pandas a partir de la matriz de booleanos

En esta parte del código, se está creando un DataFrame de pandas utilizando la matriz binaria `te_ary` generada previamente y los nombres de las columnas obtenidos de `te.columns_`.

- `te.columns_` Los nombres de columnas se obtienen del `TransactionEncoder` luego de que ha sido ajustado a las transacciones.
- `te_ary` es la matriz binaria


In [5]:
# Crear un DataFrame de pandas a partir de la matriz de booleanos
# Esto facilita el manejo de los datos en los siguientes pasos
df = pd.DataFrame(te_ary, columns=te.columns_)

# Encontrar conjuntos de elementos frecuentes con un soporte mínimo de 60%

En esta parte del código, se está aplicando el algoritmo Apriori a los datos representados en el DataFrame `df` para encontrar conjuntos de ítems frecuentes.

- `apriori` es una función que toma como entrada el DataFrame `df`.
- `min_support=0.6` En este caso, se ha establecido en 0.6, lo que significa que solo se considerarán los conjuntos de ítems que aparezcan en al menos el 60% de las transacciones.
- `use_colnames=True` indica que se utilizarán los nombres de columna originales en el DataFrame.

El resultado, `items_frecuentes`, contendrá los conjuntos de ítems que cumplen con el umbral de soporte mínimo especificado.

In [6]:
# Encontrar conjuntos de elementos frecuentes con un soporte mínimo de 60%
# Un conjunto de elementos es frecuente si aparece en al menos el 60% de las transacciones
items_frecuentes = apriori(df, min_support=0.6, use_colnames=True)

## Generar las reglas de asociación con una confianza mínima de 60%

En esta parte del código, se están generando reglas de asociación a partir de los conjuntos de ítems frecuentes utilizando la función `association_rules`.

- `items_frecuentes` es el DataFrame que contiene los conjuntos de ítems frecuentes encontrados mediante el algoritmo Apriori.
- `metric="confidence"` especifica que la métrica utilizada para evaluar las reglas de asociación será la confianza.
- `min_threshold=0.6` indica que solo se generarán reglas que tengan una confianza igual o mayor al 60%.

El resultado, `reglas`, contendrá las reglas de asociación con una confianza mínima de 60%

In [7]:
# Generar las reglas de asociación con una confianza mínima de 60%
# Una regla de asociación es una implicación de la forma {A} -> {B}
# La confianza de la regla es la proporción de transacciones con A que también contienen B
reglas = association_rules(items_frecuentes, metric="confidence", min_threshold=0.6)

## Mostrar Data Frame

In [8]:
# Convertir los valores de soporte y confianza a porcentaje y añadir el símbolo '%'
# Esto hace que los resultados sean más fáciles de interpretar
reglas['support'] = (reglas['support']*100).apply(lambda x: f'{x:.2f}%')
reglas['confidence'] = (reglas['confidence']*100).apply(lambda x: f'{x:.2f}%')

# Mostrar solo las columnas deseadas
# Estas columnas contienen la información más relevante sobre las reglas de asociación
reglas = reglas[['antecedents', 'consequents', 'support', 'confidence', 'lift' ]]
print(reglas)

       antecedents     consequents  support confidence  lift
0          (HUEVO)         (CREMA)   66.67%     66.67%   1.0
1          (CREMA)         (HUEVO)   66.67%    100.00%   1.0
2          (CREMA)         (LECHE)   66.67%    100.00%   1.0
3          (LECHE)         (CREMA)   66.67%     66.67%   1.0
4          (HUEVO)         (LECHE)  100.00%    100.00%   1.0
5          (LECHE)         (HUEVO)  100.00%    100.00%   1.0
6          (HUEVO)           (PAN)   66.67%     66.67%   1.0
7            (PAN)         (HUEVO)   66.67%    100.00%   1.0
8            (PAN)         (LECHE)   66.67%    100.00%   1.0
9          (LECHE)           (PAN)   66.67%     66.67%   1.0
10  (HUEVO, CREMA)         (LECHE)   66.67%    100.00%   1.0
11  (HUEVO, LECHE)         (CREMA)   66.67%     66.67%   1.0
12  (CREMA, LECHE)         (HUEVO)   66.67%    100.00%   1.0
13         (HUEVO)  (CREMA, LECHE)   66.67%     66.67%   1.0
14         (CREMA)  (HUEVO, LECHE)   66.67%    100.00%   1.0
15         (LECHE)  (HUE

## 1. **Importación de bibliotecas:**
   - `from itertools import combinations, chain`: Importa las funciones `combinations` y `chain` del módulo `itertools`. Estas funciones se utilizan para generar combinaciones y permutaciones.

In [12]:
from itertools import combinations, chain

In [ ]:

# Conjunto de datos
dataset = [
    ("LECHE", "PAN", "HUEVO"),
    ("CEREAL", "LECHE", "CREMA", "HUEVO"),
    ("LECHE", "CREMA", "PAN", "HUEVO")
]

# Función para generar subconjuntos de un conjunto
def subsets(arr):
    return chain(*[combinations(arr, i + 1) for i, a in enumerate(arr)])

# Función para obtener conjuntos de elementos frecuentes
def get_frequent_itemsets(transactions, min_support):
    counter = {}
    for transaction in transactions:
        subsets_list = list(map(frozenset, subsets(transaction)))
        for subset in subsets_list:
            if subset not in counter:
                counter[subset] = 1
            else:
                counter[subset] += 1

    return {itemset: support for itemset, support in counter.items() if support/len(transactions) >= min_support}

# Función para calcular el soporte, la confianza y el lift
def calculate_rules(frequent_itemsets, min_confidence, total_transactions):
    rules = []
    for itemset in frequent_itemsets:
        for subset in subsets(itemset):
            subset = frozenset(subset)
            diff = itemset.difference(subset)
            if diff:
                support_itemset = frequent_itemsets[itemset] / total_transactions
                support_subset = frequent_itemsets[subset] / total_transactions
                support_diff = frequent_itemsets[diff] / total_transactions
                confidence = frequent_itemsets[itemset] / frequent_itemsets[subset]
                lift = confidence / support_diff
                if confidence >= min_confidence:
                    rules.append((subset, diff, support_itemset, confidence, lift))
    return rules

# Obtener conjuntos de elementos frecuentes
frequent_itemsets = get_frequent_itemsets(dataset, min_support=0.6)

# Calcular reglas
rules = calculate_rules(frequent_itemsets, min_confidence=0.6, total_transactions=len(dataset))

# Crear DataFrame
rules_df = pd.DataFrame(rules, columns=['Antecedente', 'Consecuente', 'Soporte', 'Confianza', 'Lift'])

# Convertir conjuntos a listas para una mejor visualización
rules_df['Antecedente'] = rules_df['Antecedente'].apply(list)
rules_df['Consecuente'] = rules_df['Consecuente'].apply(list)

# Convertir soporte y confianza a porcentajes con dos decimales
rules_df['Soporte'] = rules_df['Soporte'].apply(lambda x: f'{x * 100:.2f}%')
rules_df['Confianza'] = rules_df['Confianza'].apply(lambda x: f'{x * 100:.2f}%')
rules_df['Lift'] = rules_df['Lift'].apply(lambda x: f'{x * 100:.2f}%')

# Mostrar DataFrame
print(rules_df)